<a href="https://colab.research.google.com/github/Guest838/ML_Course-3.0/blob/main/06A_hw_Kai_Asano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import transforms


In [ ]:
fil = pd.read_csv("drive/MyDrive/filt/algos.csv")
fil

,-1.0,-0.5,0.0,-0.5.1,0.5,0.5.1,0.0.1,0.5.2,1.0
0,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625


In [ ]:
    self.filt2.weight.data = ([[-1.0, -0.5, 0.0],
      [-0.5, 0.5, 0.5],
      [0.0, 0.5, 1.0]])
    self.filt1.weight.data = ([[0.0625, 0.0625, 0.0625],
      [0.0625, 0.0625, 0.0625],
      [0.0625, 0.0625, 0.0625]])

In [ ]:
class Net(nn.Module):

    def __init__(self):
      super(Net, self).__init__()
      self.filt1 = nn.Conv2d(1, 1, 3, padding=1, bias=False)
      kernel = torch.FloatTensor([[[[0.0625, 0.0625, 0.0625],
                                     [0.0625, 0.0625, 0.0625],
                                     [0.0625, 0.0625, 0.0625]]]])
      self.filt1.weight.data = nn.Parameter(kernel)
      self.filt2 = nn.Conv2d(1, 1, 3, padding=1, bias=False)
      self.filt3 = nn.Conv2d(1, 1, 3, padding=1, bias=False)
      kernel3 = torch.FloatTensor([[[[-1.0, -0.5, 0.0],
      [-0.5, 0.5, 0.5],
      [0.0, 0.5, 1.0]]]])
      self.filt3.weight.data = nn.Parameter(kernel3)

    def forward(self, x):

        x = self.filt1(x)
        x = self.filt2(x)
        x = self.filt3(x)
        x = x.flatten()
        return x




In [ ]:
class DatasetCats(Dataset):
    def __init__(self, transform=None):
      self.transform = transform
      self.labels = [None] * 1000
      self.data= [None] * 1000
      for i in range(1, 1001):
        label = np.loadtxt(f"drive/MyDrive/filt/{i}.txt")
        label = np.array(label, dtype = np.float32).reshape(4096)
        label = torch.from_numpy(label)
        self.labels[i-1] = label
        image =  torchvision.io.read_image(f"drive/MyDrive/filt/{i}.png")
        image = np.array(image, dtype = np.float32)
        image = torch.from_numpy(image)
        self.data[i-1] = image
        print(i)


    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (C, H, W) --> (H, W, C)
        image = self.data[index]
        label = self.labels[index]

        if self.transform is not None:
            image = self.transform(image)

        return image, label

In [ ]:
full_dataset = DatasetCats(transform=None)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
img, lab = full_dataset.__getitem__(0)

print(img.shape)
print(type(img))
print(lab.shape)

torch.Size([1, 64, 64])
<class 'torch.Tensor'>
torch.Size([4096])


In [ ]:
train_loader = DataLoader(full_dataset, batch_size=8, shuffle=True, num_workers=1)

In [ ]:
newNet=Net()
newNet.filt1.requires_grad_(False)
newNet.filt3.requires_grad_(False)

Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [ ]:
import torchsummary
torchsummary.summary(newNet, (1,64,64))

In [ ]:
device = (
    torch.device(f"cuda:{0}") if torch.cuda.is_available() else torch.device("cpu")
)
newNet.to(device)

Net(
  (filt1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (filt2): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (filt3): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)

In [ ]:
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(newNet.parameters(), lr=1e-4)
epochs = 1000
for epoch in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.cuda(), labels.cuda()
        outputs = newNet(images)
        loss = loss_fn(outputs, labels.reshape(32768))


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

Epoch 1/1000, Loss: 17153.538765625
Epoch 2/1000, Loss: 15844.9241484375
Epoch 3/1000, Loss: 14614.09039453125
Epoch 4/1000, Loss: 13457.5758359375
Epoch 5/1000, Loss: 12370.257578125
Epoch 6/1000, Loss: 11347.69204296875
Epoch 7/1000, Loss: 10388.53192578125
Epoch 8/1000, Loss: 9490.85166796875
Epoch 9/1000, Loss: 8651.20825390625
Epoch 10/1000, Loss: 7867.25477734375
Epoch 11/1000, Loss: 7137.14669921875
Epoch 12/1000, Loss: 6457.72721484375
Epoch 13/1000, Loss: 5827.57997265625
Epoch 14/1000, Loss: 5243.35338671875
Epoch 15/1000, Loss: 4703.848654296875
Epoch 16/1000, Loss: 4206.773046875
Epoch 17/1000, Loss: 3750.07569921875
Epoch 18/1000, Loss: 3332.00990234375
Epoch 19/1000, Loss: 2950.0762841796877
Epoch 20/1000, Loss: 2602.977619140625
Epoch 21/1000, Loss: 2289.0684326171877
Epoch 22/1000, Loss: 2005.64197265625
Epoch 23/1000, Loss: 1751.520107421875
Epoch 24/1000, Loss: 1524.7295263671874
Epoch 25/1000, Loss: 1323.6061225585938
Epoch 26/1000, Loss: 1146.1124462890625
Epoch 27/

In [ ]:
print(newNet.filt1.weight)

Parameter containing:
tensor([[[[0.0625, 0.0625, 0.0625],
          [0.0625, 0.0625, 0.0625],
          [0.0625, 0.0625, 0.0625]]]], device='cuda:0')


In [ ]:
print(newNet.filt2.weight)

Parameter containing:
tensor([[[[0.1250, 0.2500, 0.1250],
          [0.2500, 0.5000, 0.2500],
          [0.1250, 0.2500, 0.1250]]]], device='cuda:0', requires_grad=True)


In [ ]:
print(newNet.filt3.weight)

Parameter containing:
tensor([[[[-1.0000, -0.5000,  0.0000],
          [-0.5000,  0.5000,  0.5000],
          [ 0.0000,  0.5000,  1.0000]]]], device='cuda:0')
